In [1]:
!pip install firebase_admin

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
pip install konlpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 46.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 kB 26.5 MB/s eta 0:00:00


In [3]:
pip install git+https://github.com/linto-ai/whisper-timestamped

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/linto-ai/whisper-timestamped to /tmp/pip-req-build-92mekexl
  Running command git clone --filter=blob:none --quiet https://github.com/linto-ai/whisper-timestamped /tmp/pip-req-build-92mekexl
  Resolved https://github.com/linto-ai/whisper-timestamped to commit 2c55305d6aa53f0c0fa1fe63fc85c33bfa60e963
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 645.5/645.5 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 792.9/792.9 kB 40.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 59.3 MB/s eta 0:00:00
  Created wheel for whisper-timestamped: filename=whisper_timestamped-1.12.20-py3-none-any.whl size=44514 sha256=6911e3d809c22c2e2b6335b21e8eb03be84af4d11

In [4]:
pip install nextpow2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement nextpow2 (from versions: none)
ERROR: No matching distribution found for nextpow2


In [5]:
import firebase_admin
from firebase_admin import credentials
from firebase_admin import storage
from firebase_admin import db
import whisper_timestamped as whisper
import pandas as pd
import json
import librosa
import time


# Firebase Admin SDK 인증 정보 로드
cred = credentials.Certificate('/content/speechking-787ec-firebase-adminsdk-6mz9r-00885bb0e1.json')  # 서비스 계정 키 파일 경로
firebase_admin.initialize_app(cred, {'storageBucket': 'speechking-787ec.appspot.com',
                                     'databaseURL' : 'https://speechking-787ec-default-rtdb.firebaseio.com/'})  # 스토리지 버킷 이름

Importing the dtw module. When using in academic works please cite:
  T. Giorgino. Computing and Visualizing Dynamic Time Warping Alignments in R: The dtw Package.
  J. Stat. Soft., doi:10.18637/jss.v031.i07.



In [6]:
keyword_ref = db.reference("keyword_list")  #대본 분석에서 강조된 단어 리스트 가져옴

data = keyword_ref.get()

# Firebase Realtime Database의 'filename' 참조
filename_ref = db.reference('filename')

# 이전 파일 경로 저장 변수
previous_filename = None

# 다운로드 및 결과 출력 함수
def download_and_transcribe_file(file_path):
    # Storage에서 파일 다운로드
    bucket = storage.bucket()
    blob = bucket.blob(file_path)
    output_file_path = "/content/drive/MyDrive/record.wav"  # 저장할 파일의 경로와 파일명
    time.sleep(2)
    blob.download_to_filename(output_file_path)

    audio = whisper.load_audio("/content/drive/MyDrive/record.wav")  #가져온 파일을 변수로 저장
    model = whisper.load_model("tiny", device="cpu")
    result = whisper.transcribe(model, audio, language="ko")

    df = pd.concat([pd.DataFrame(segment['words']) for segment in result['segments']])
    df = df.reset_index(drop=True)

    y, sr = librosa.load("/content/drive/MyDrive/record.wav", sr=None)

    keyword_list = set(data)

   # 높낮이 추출 함수
    def extract_pitch(start_time, end_time):
        start_sample = int(start_time * sr)
        end_sample = int(end_time * sr)
        y_segment = y[start_sample:end_sample]
        pitches, magnitudes = librosa.piptrack(y=y_segment, sr=sr)
        mean_pitches = []
        for t in range(pitches.shape[1]):
            nonzero_pitches = pitches[:,t][magnitudes[:,t] > 0]
            if len(nonzero_pitches) > 0:
                mean_pitches.append(nonzero_pitches.mean())
            else:
                mean_pitches.append(0)
        return mean_pitches


    df['pitch'] = df.apply(lambda row: extract_pitch(row['start'], row['end']), axis=1)

     # 높낮이 추출 함수2, 세기 추출 함수
    def extract_features(start_time, end_time):
        start_sample = int(start_time * sr)
        end_sample = int(end_time * sr)
        y_segment = y[start_sample:end_sample]
        pitches, magnitudes = librosa.piptrack(y=y_segment, sr=sr)
        nonzero_pitches = pitches[magnitudes > 0]
        if nonzero_pitches.size > 0:
            mean_pitch = nonzero_pitches.mean()
        else:
            mean_pitch = 0
    
        rms_amplitude = librosa.feature.rms(y=y_segment)
        intensity = rms_amplitude.mean()
    
        return mean_pitch, intensity

    df[['pitch', 'intensity']] = df.apply(lambda row: pd.Series(extract_features(row['start'], row['end'])), axis=1)

    new_df = pd.DataFrame(columns=['text', 'pitch', 'intensity'])

   #녹음 단어와 강조 단어 일치하는지 파악
    for index, row in df.iterrows():
      found_keywords = set()
      for keyword in keyword_list:
          if keyword in row['text']:
              found_keywords.add(keyword)
        
      if found_keywords:
          
          keyword_list -= found_keywords
          new_df = new_df.append(row)

    new_df = new_df.reset_index(drop=True)

    score_p= 0
    score_i=0

   # 높낮이 점수
    for index, row in new_df.iterrows():
        if row['pitch'] > 750:
            score_p += 5/len(new_df)

    if 0.0<=score_p<1.0:
      score_p= 0
      message_p='제발....연습하세요.'
    if score_p==1:
      score_p=1
      message_p="강조가 전혀 되지 않았어요.\n더 많은 연습이 필요해요."
    if 1.1<=score_p<2.0:
      score_p= 1.5
      message_p="강조가 많이 부족해요.\n연습을 더 해보세요."
    if score_p==2:
      score_p=2
      message_p="강조가 부족한 편이에요.\n좀 더 신경써서 연습해보세요."
    if 2.1<=score_p<3.0:
      score_p= 2.5
      message_p="약간의 강조가 있지만 아직 부족해요.\n계속 연습해보세요."
    if score_p==3:
      score_p=3
      message_p= "강조는 있지만 아직 부족해요.\n조금 더 높게 해보세요."
    if 3.1<=score_p<4.0:
      score_p= 3.5
      message_p="일부 강조가 보이지만 더 발전할 수 있어요.\n꾸준한 연습이 필요해요."
    if score_p==4:
      score_p=4
      message_p='잘했습니다.\n한번 더 연습해볼까요?'
    if 4.1<=score_p<5.0:
      score_p= 4.5
      message_p="강조가 잘 되고 있어요!\n조금만 더 높하면 완벽할 거예요."
    if score_p==5.0:
      score_p= 5
      message_p="강조가 완벽해요!\n훌륭한 성과를 보여주고 있어요."


   # 세기 점수
    for index, row in new_df.iterrows():
        if row['intensity'] > 0.04:
            score_i += 5/len(new_df)

    if 0.0<=score_i<1.0:
      score_i= 0
      message_i='크게 말하는 연습이 필요합니다...\n조용한 곳에서 녹음해주세요'
    if score_i==1:
      score_i=1
      message_i="소리가 매우 약해요.\n더 명확하고 크게 발성해보세요."
    if 1.1<=score_i<2.0:
      score_i= 1.5
      message_i="소리가 소리가 약간 부족해요.\n발성을 좀 더 선명하게 해보세요."
    if score_i==2:
      score_i=2
      message_i="소리가 약해요.\n더 크게 발성해보세요."
    if 2.1<=score_i<3.0:
      score_i= 2.5
      message_i='조금씩 성장 중...\n강하게 플리즈'
    if score_i==3:
      score_i=3
      message_i="소리는 보통 수준이지만,\n더 크게 발성하는 게 도움이 될 거예요."
    if 3.1<=score_i<4.0:
      score_i= 3.5
      message_i="소리가 보통 수준이에요.\n좀 더 선명하고 강하게 발성해보세요."
    if score_i==4:
      score_i=4
      message_i="좋아요!\n발성이 명확하고 들리기 쉬워요."
    if 4.1<=score_i<5.0:
      score_i= 4.5
      message_i="매우 좋아요!\n명확하고 강력하게 발성하고 계시네요."
    if score_i==5.0:
      score_i= 5
      message_i="소리가 완벽해요!\n선명하고 강력한 목소리로 잘 발성하셨어요."

    print(message_p)
    print(message_i)

    mp_reference = db.reference('sp')
    mp_reference.set(score_p)

    mi_reference = db.reference('si')
    mi_reference.set(score_i)
    
    mp_reference = db.reference('mp')
    mp_reference.set(message_p)

    mi_reference = db.reference('mi')
    mi_reference.set(message_i)


    # 모든 행과 열이 표시되도록 설정
    #pd.set_option('display.max_rows', None)
    #pd.set_option('display.max_columns', None)

    return df
    '''print("다운로드 완료!")

    model = whisper.load_model("base")
    result = model.transcribe(output_file_path)
    print(result["text"])'''

# 무한루프로 실행하여 변경 사항 감지
while True:
    # filename 경로의 데이터 가져오기

    filename_ref = db.reference('filename')
    filename = filename_ref.get()

    # 이전 파일 경로와 현재 파일 경로 비교
    if filename != previous_filename:
        # 파일 경로가 변경되었을 경우에만 실행
        file_path = filename
        # Storage에서 파일 다운로드 및 결과 출력
        df = download_and_transcribe_file(file_path)
        print(df)
        previous_filename = filename

    # 잠시 대기
    time.sleep(1)  # 필요한 경우 대기 시간 조정

100%|█████████████████████████████████████| 72.1M/72.1M [00:00<00:00, 94.1MiB/s]
100%|██████████| 430/430 [00:01<00:00, 246.08frames/s]
<ipython-input-6-1cb725b5a957>:29: UserWarning: PySoundFile failed. Trying audioread instead.
  y, sr = librosa.load("/content/drive/MyDrive/record.wav", sr=None)
/usr/local/lib/python3.10/dist-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)
<ipython-input-6-1cb725b5a957>:79: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append(row)


제발....연습하세요.
소리가 완벽해요!
선명하고 강력한 목소리로 잘 발성하셨어요.
    text  start   end  confidence       pitch  intensity
0    내용과   1.22  1.70       0.477  441.602692   0.076261
1   상관없는   1.70  2.40       0.673  346.252777   0.058137
2     녹음   2.40  2.78       0.843  330.870026   0.050044
3     진행   2.78  3.04       0.863  222.262085   0.058697
4  중입니다.   3.04  3.74       0.867  298.424194   0.028594


KeyboardInterrupt: ignored